# Importing libraries

In [2]:
import pandas as pd
import numpy as np
import pathlib

# Loading data

In [10]:
pathlib.Path("../Population based MPCE/MPCE_1Jan24.pkl").resolve().exists()

True

In [ ]:
pp_path = r"../Population based MPCE/MPCE_1Jan24.pkl"

In [34]:
#Reading the pickle file
df_pp = pd.read_pickle(pp_path)

In [35]:
df_pp.head()

,food_total,consumables_total,durables_total,total expenditure,hh_size,sector,multiplier,mpce,normal_multiplier,cum_normal_multiplier,decile
hhid,,,,,,,,,,,
31000301,8704.43,9655.57,1315.64,19675.64,1.0,2,96498,19675.64,0.00028,98.070289,10
31000302,13032.43,24088.47,3176.71,40297.61,5.0,2,96498,8059.52,0.00028,77.872489,8
31000303,13252.86,21439.29,3447.95,38140.10,5.0,2,96498,7628.02,0.00028,74.870871,8
31000304,5644.29,5560.11,1840.85,13045.25,2.0,2,96498,6522.62,0.00028,65.185410,7
31000305,1263.14,4418.52,1084.11,6765.77,1.0,2,96498,6765.77,0.00028,67.463181,7


In [36]:
#Keeping only relavant columns
df_pp = df_pp[['food_total', 'consumables_total', 'durables_total', 'total expenditure', 'sector', 'multiplier']]

In [37]:
#Sorting based on total expenditure
df_pp = df_pp.sort_values(by = 'total expenditure')
df_pp.head()

,food_total,consumables_total,durables_total,total expenditure,sector,multiplier
hhid,,,,,,
69894311,0.00,185.00,0.00,185.00,1,259645
33802314,0.00,0.00,291.78,291.78,2,130882
67452312,111.43,173.23,51.78,336.44,1,215726
66357307,0.00,186.44,204.25,390.69,1,67519
69148311,150.00,144.73,100.27,395.00,1,228600


In [38]:
#Converting monthly expenditure to yearly values
df_pp[['food_total', 'consumables_total', 'durables_total', 'total expenditure']] = df_pp[['food_total', 'consumables_total', 'durables_total', 'total expenditure']] * (365/30)

#Rounding expenditure to 2 digits
df_pp[['food_total', 'consumables_total', 'durables_total', 'total expenditure']] = df_pp[['food_total', 'consumables_total', 'durables_total', 'total expenditure']].map(lambda x: np.round(x,2))
df_pp.head()

,food_total,consumables_total,durables_total,total expenditure,sector,multiplier
hhid,,,,,,
69894311,0.00,2250.83,0.00,2250.83,1,259645
33802314,0.00,0.00,3549.99,3549.99,2,130882
67452312,1355.73,2107.63,629.99,4093.35,1,215726
66357307,0.00,2268.35,2485.04,4753.39,1,67519
69148311,1825.00,1760.88,1219.95,4805.83,1,228600


In [39]:
#Assigning deciles to HHs based on total expenditure
df_pp['hh_decile'] = pd.qcut(x = df_pp['total expenditure'],
                             q = 10,
                             labels = range(1,11)
                             )

In [40]:
def wt_avg (group):
        table = group[['food_total', 'consumables_total', 'durables_total', 'total expenditure', 'multiplier']]
        weighted_average = {}
        for col in ['food_total', 'consumables_total', 'durables_total', 'total expenditure']:
            valid_data = table[[col,'multiplier']]
            sum_product = (valid_data[col]*valid_data['multiplier']).sum()
            wt_sum = valid_data['multiplier'].sum()

            weighted_average[col] = sum_product/wt_sum if wt_sum>0 else np.nan
        return pd.Series(weighted_average)

In [41]:
try:
    display(df_pp.groupby('hh_decile', observed = False).apply(wt_avg).round(2).transpose())
except Exception as e:
    print(e)

C:\Users\Tarun\AppData\Local\Temp\ipykernel_28372\3445013436.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  display(df_pp.groupby('hh_decile', observed = False).apply(wt_avg).round(2).transpose())


hh_decile,1,2,3,4,5,6,7,8,9,10
food_total,38396.05,59880.21,73406.58,84740.29,95672.03,106993.50,118631.58,133763.30,154396.85,211847.05
consumables_total,28288.83,46472.52,58515.31,70253.90,82901.55,96499.37,114344.04,137918.29,174945.04,298767.25
durables_total,7254.97,12687.32,16139.91,19471.39,22789.72,26973.88,32353.61,38296.92,49384.90,102835.13
total expenditure,73939.85,119040.05,148061.80,174465.58,201363.30,230466.75,265329.24,309978.51,378726.80,613449.42


# Saving

In [42]:
df_pp[['total expenditure', 'multiplier', 'hh_decile']].to_pickle("hh_decile.pkl")